In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [2]:
df = pd.read_csv(r'C:\Users\VictorAdeife\Downloads\energydata_complete.csv')
df.head(2)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.7900,19.79,44.73,19.0,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.7225,19.79,44.79,19.0,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195


In [3]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two D.P?



In [4]:
T2 = df.T2
T6 = df.T6
r_squared = r2_score(T2, T6)
np.round(r_squared, 2)

-35.39

What is the Mean Absolute Error (in two decimal places)?

In [5]:
mae = mean_absolute_error(T2, T6)
np.round(mae, 2)

12.44

What is the Residual Sum of Squares (in two decimal places)?

In [6]:
rss = np.sum(np.square(T2 - T6))
np.round(rss, 2)

3453843.04

What is the Root Mean Squared Error (in three decimal places)?

In [7]:
mse = mean_squared_error(T2, T6, squared=False)
np.round(mse, 3)

13.229

What is the Coefficient of Determination (in two decimal places)?

In [8]:
r_squared = r2_score(T2, T6)
np.round(r_squared, 2)

-35.39

In [9]:
train_a = df.drop(['lights', 'date'], 1)
label = df.Appliances

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scalar = MinMaxScaler()

In [12]:
train = scalar.fit_transform(train_a)

In [13]:
train = pd.DataFrame(train, columns=train_a.columns)

In [14]:
label = train.Appliances

In [15]:
train = train.drop(['Appliances'], 1)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(train, label, test_size=0.30, random_state=42)

In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
model = LinearRegression()

In [20]:
model.fit(X_train, y_train)

LinearRegression()

In [41]:
coefficient = pd.DataFrame(model.coef_, X_train.columns)
coefficient

,0
T1,-0.003281
RH_1,0.553547
T2,-0.236178
RH_2,-0.456698
T3,0.290627
RH_3,0.096048
T4,0.028981
RH_4,0.026386
T5,-0.015657
RH_5,0.016006


In [22]:
print(coefficient.min())
print(coefficient.max())

0   -0.456698
dtype: float64
0    0.553547
dtype: float64


In [23]:
from sklearn.linear_model import ridge_regression, Ridge, Lasso

Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [24]:
ridge = Ridge(alpha=0.4)

In [25]:
ridge.fit(X_train, y_train)

Ridge(alpha=0.4)

In [26]:
train_pred = ridge.predict(X_train)

In [27]:
train_mse = mean_squared_error(y_train, train_pred, squared=False)
train_mse

0.08899770418364146

In [28]:
test_pred = ridge.predict(X_test)

In [29]:
test_mse = mean_squared_error(y_test, test_pred, squared=False)
test_mse

0.08753385704628003

In [33]:
train_mse == test_mse

False

In [34]:
lasso = Lasso(alpha=0.001)
lasso.fit(X_train, y_train)

Lasso(alpha=0.001)

In [35]:
train_pred_a = lasso.predict(X_train)
mean_squared_error(y_train, train_pred_a, squared=False)

0.09501109253338288

In [36]:
test_pred_a = lasso.predict(X_test)
mean_squared_error(y_test, test_pred_a, squared=False)

0.09358170467245137

In [42]:
l_coef = pd.DataFrame(lasso.coef_, X_test.columns)
l_coef

,0
T1,0.000000
RH_1,0.017880
T2,0.000000
RH_2,-0.000000
T3,0.000000
RH_3,0.000000
T4,-0.000000
RH_4,0.000000
T5,-0.000000
RH_5,0.000000


In [46]:
l_coef[l_coef != 0].count()

0    4
dtype: int64